In [31]:
print("Hello LangChain!!")

Hello LangChain!!


In [32]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

#load_dotenv(dotenv_path='.env')
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])


gsk_E


In [33]:
# prompt + llm + output 

# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("user", "{input}") ]
)
print(type(prompt))
print(prompt)

<class 'langchain_core.prompts.chat.ChatPromptTemplate'>
input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [34]:
prompt_text = prompt.format(input="파이썬은 무엇인가요? 자세하게 설명해주세요")
print(type(prompt_text))
print(prompt_text)

<class 'str'>
System: 당신은 개발자입니다.
Human: 파이썬은 무엇인가요? 자세하게 설명해주세요


In [35]:
#llm = ChatOpenAI(api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo-0125")

# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    #model="moonshotai/kimi-k2-instruct-0905",
    model="openai/gpt-oss-120b",
    temperature=0.7
)
print(type(llm))
print(llm)

<class 'langchain_openai.chat_models.base.ChatOpenAI'>
client=<openai.resources.chat.completions.completions.Completions object at 0x00000202F8225940> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000202F8225C70> root_client=<openai.OpenAI object at 0x00000202F8224950> root_async_client=<openai.AsyncOpenAI object at 0x00000202F8225640> model_name='openai/gpt-oss-120b' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


In [36]:
try:
    response = llm.invoke(prompt_text)
    print(type(response))
    print("응답:", response.content)
except Exception as e:
    print(f"오류 발생: {e}")

오류 발생: Error code: 400 - {'error': {'message': 'Organization has been restricted. Please reach out to support if you believe this was in error.', 'type': 'invalid_request_error', 'code': 'organization_restricted'}}


# LCEL

In [37]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [38]:
chain = prompt | llm | output_parser
print(type(chain))
print(chain)

<class 'langchain_core.runnables.base.RunnableSequence'>
first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]) middle=[ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000202F8225940>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000202F8225C70>, root_client=<openai.OpenAI object at 0x00000202F8224950>, root_async_client=<openai.AsyncOpenAI object at 0x00000202F8225640>, model_name='openai/gpt-oss-120b', temperature=0.7, model_kwargs={}, openai_api_key=SecretStr('**********'), openai_api_base='https://api.groq.com/openai/v1')] last=StrOutputPars

In [39]:
response = chain.invoke({"input":"LangChain은 무엇인가요?"})

BadRequestError: Error code: 400 - {'error': {'message': 'Organization has been restricted. Please reach out to support if you believe this was in error.', 'type': 'invalid_request_error', 'code': 'organization_restricted'}}

In [ ]:
print(type(response))
print(response)

<class 'str'>
## LangChain이란?

**LangChain**은 대형 언어 모델(LLM, Large Language Model)을 **애플리케이션 수준**으로 활용할 수 있게 해 주는 **오픈소스 프레임워크**입니다.  
LLM을 단순히 “텍스트를 생성하는 엔진”으로 쓰는 것이 아니라, **데이터베이스, API, 파일 시스템, 사용자 인터페이스 등 다양한 외부 자원**과 결합해 **복합적인 워크플로우**를 만들 수 있도록 도와줍니다.

---

## 핵심 개념

| 개념 | 설명 | 예시 |
|------|------|------|
| **Chain** | 여러 LLM 호출·프롬프트·처리 단계를 **순차·조건부·반복**으로 연결한 흐름 | 질문 → 검색 → 요약 → 답변 |
| **PromptTemplate** | 변수(`{question}` 등)를 삽입해 동적으로 프롬프트를 생성 | `"다음 질문에 답해 주세요: {question}"` |
| **LLM** | 실제 텍스트를 생성하는 모델 (OpenAI GPT‑4, Anthropic Claude, Llama 등) | `OpenAIChat`, `ChatAnthropic` |
| **Memory** | 대화·작업 이력을 **상태**로 저장해 다음 호출에 활용 | 대화 기록, 작업 진행 상황 |
| **Agents** | **툴(tool)**(검색 엔진, 데이터베이스, 코드 실행 등)을 **동적으로 선택**해 작업 수행 | “날씨를 알려줘” → 검색 툴 호출 → 결과 반환 |
| **Tools** | 외부 서비스·함수·API를 래핑한 **플러그인** | `SerpAPI`, `SQLDatabase`, `PythonREPL` |
| **Retriever** | 텍스트/문서 컬렉션에서 **관련 문서**를 검색해 LLM에 제공 | 벡터 DB(FAISS, Pinecone) 기반 검색 |
| **Callbacks** | 실행 흐름을 **감시·로깅·시각화**할 수 있는 훅 | LangSmith(모니터링), S